In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
population_data = pd.read_csv('../data/Population_E_All_Data_NOFLAG.csv', encoding = "ISO-8859-1")

In [3]:
element_code_map = population_data[['Element Code', 'Element']].value_counts().reset_index(name='count')
element_code_map = element_code_map.sort_values(by=['Element Code'])[['Element Code', 'Element']].reset_index(drop=True)
area_code_map = population_data[['Area Code', 'Area']].value_counts().reset_index(name='count')
area_code_map = area_code_map.sort_values(by=['Area Code'])[['Area Code', 'Area']].reset_index(drop=True)

In [4]:
def get_list_of_countries():
    countries = area_code_map[area_code_map['Area Code'] < 1000]
    # Removing China because it's an aggregation of multiple entries.
    countries = countries[countries['Area'] != 'China'].reset_index(drop=True)
    return countries

In [5]:
countries = get_list_of_countries()

In [6]:
filtered_population_data = population_data[
    (population_data['Area Code'].isin(get_list_of_countries()['Area Code']))
    & (population_data['Element Code'] == 511)
].reset_index(drop=True)
del population_data

In [7]:
mega_dataset = filtered_population_data.drop(['Item Code', 'Item'], axis=1)

In [8]:
climate_data = pd.read_csv('../data/Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding = "ISO-8859-1")

In [9]:
filtered_climate_data = climate_data[
    (climate_data['Months Code'] == 7020)
    & (climate_data['Element Code'] == 7271)
    & (climate_data['Area Code'].isin(countries['Area Code']))
].reset_index(drop=True)
del climate_data

In [10]:
filtered_climate_data = filtered_climate_data.drop(['Months Code', 'Months'], axis=1)

In [11]:
mega_dataset = mega_dataset.append(
    filtered_climate_data
).sort_values(
    by=['Area Code', 'Element']
).reset_index(drop=True)

In [12]:
food_security_data = pd.read_csv('../data/Food_Security_Data_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [13]:
filtered_food_security_data = food_security_data[
    (food_security_data['Area Code'].isin(countries['Area Code']))
    & (food_security_data['Item Code'].isin([210041, 210011, 210401, 210071, 210091, 210081]))
]
del food_security_data


In [14]:
cols_to_process = [x for x in filtered_food_security_data.columns if x.startswith('Y2') and len(x) > 5]
for col in cols_to_process:
    year = 'Y' + str(int(col[5:]) - 1)
    filtered_food_security_data.loc[:,year] = filtered_food_security_data.loc[:, col]
filtered_food_security_data = filtered_food_security_data.drop(cols_to_process, axis=1)

In [15]:
filtered_food_security_data['Element Code'] = filtered_food_security_data['Item Code']
filtered_food_security_data['Element'] = filtered_food_security_data['Item']
filtered_food_security_data = filtered_food_security_data.drop(['Item', 'Item Code'], axis=1)

In [16]:
filtered_food_security_data

,Area Code,Area,Element Code,Element,Unit,Y2000,Y2001,Y2002,Y2003,Y2004,...,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
7,2,Afghanistan,210041,Prevalence of undernourishment (percent) (3-ye...,%,NaN,47.8,45.6,40.6,38,...,24.7,28.2,26.3,24.2,21.5,22.2,23,23.4,25.6,NaN
8,2,Afghanistan,210011,Number of people undernourished (million) (3-y...,millions,NaN,10.3,10.3,9.6,9.4,...,7.4,8.8,8.5,8.1,7.4,7.8,8.3,8.7,9.7,NaN
9,2,Afghanistan,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.8,15.1,17.3,17.3,19.8,NaN
10,2,Afghanistan,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,45.1,49.6,54.3,57.5,63.1,NaN
11,2,Afghanistan,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.1,5.3,6.3,6.4,7.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7333,181,Zimbabwe,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7334,181,Zimbabwe,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,35.5,36.7,35.9,34.2,32.1,NaN
7335,181,Zimbabwe,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,64.7,66.7,67,66.7,69.8,NaN
7336,181,Zimbabwe,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.9,5.2,5.1,4.9,4.7,NaN


In [17]:
mega_dataset = mega_dataset.append(filtered_food_security_data)

In [18]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2091,Y2092,Y2093,Y2094,Y2095,Y2096,Y2097,Y2098,Y2099,Y2100
0,1,Armenia,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Armenia,511,Total Population - Both sexes,1000 persons,NaN,NaN,NaN,NaN,NaN,...,2175.894,2160.763,2145.660,2130.563,2115.465,2100.332,2085.130,2069.810,2054.326,2038.611
2,2,Afghanistan,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Afghanistan,511,Total Population - Both sexes,1000 persons,7752.118,7840.156,7935.997,8039.694,8151.317,...,76578.450,76448.747,76304.424,76146.060,75974.256,75789.684,75593.009,75384.981,75166.352,74937.964
4,3,Albania,7271,Temperature change,°C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7333,181,Zimbabwe,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7334,181,Zimbabwe,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7335,181,Zimbabwe,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7336,181,Zimbabwe,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Removing countries that do not have the full data
mega_dataset = mega_dataset[
    (mega_dataset['Area Code'].isin(filtered_food_security_data['Area Code'].unique()))
    & (mega_dataset['Area Code'].isin(filtered_climate_data['Area Code'].unique()))
].sort_values(
    by=['Area Code', 'Element']
).reset_index(drop=True)

countries = countries[countries['Area Code'].isin(mega_dataset['Area Code'].unique())]

In [20]:
# Removing projected and future data
unnecessary_cols = [x for x in mega_dataset.columns if x.startswith('Y2') and int(x[1:]) > 2020]
mega_dataset = mega_dataset.drop(unnecessary_cols, axis=1)

In [21]:
food_balance_data_old = pd.read_csv('../data/FoodBalanceSheetsHistoric_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")
food_balance_data_new = pd.read_csv('../data/FoodBalanceSheets_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [22]:
# Taking the columns Food Suppy, Food Suppy Quantity, Losses respectively
filtered_food_balance_data_old = food_balance_data_old[
    (food_balance_data_old['Area Code'].isin(countries['Area Code']))
    & (food_balance_data_old['Element Code'].isin([664, 645, 5123]))
]

filtered_food_balance_data_new = food_balance_data_new[
    (food_balance_data_new['Area Code'].isin(countries['Area Code']))
    & (food_balance_data_new['Element Code'].isin([664, 645, 5123]))
]


In [23]:
filtered_food_balance_data_old = filtered_food_balance_data_old.drop(['Y2010', 'Y2011', 'Y2012', 'Y2013'], axis=1)

In [24]:
filtered_food_balance_data = pd.merge(
    left=filtered_food_balance_data_old,
    right=filtered_food_balance_data_new,
    how='outer',
    on=['Area Code', 'Element Code', 'Area', 'Item Code', 'Item', 'Element', 'Unit']
).reset_index(drop=True)

In [25]:
filtered_food_balance_data

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,2917.0,2698.00,...,2170.00,2152.00,2159.00,2196.00,2265.00,2250.00,2228.00,2303.00,2270.00,2273.00
1,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,2672.0,2438.00,...,1964.00,1953.00,1955.00,1993.00,2019.00,2038.00,2024.00,2108.00,2081.00,2087.00
2,2,Afghanistan,2941,Animal Products,664,Food supply (kcal/capita/day),kcal/capita/day,247.00,245.0,260.00,...,206.00,200.00,204.00,204.00,246.00,211.00,204.00,195.00,189.00,187.00
3,2,Afghanistan,2905,Cereals - Excluding Beer,5123,Losses,1000 tonnes,309.00,307.0,287.00,...,837.00,647.00,911.00,936.00,963.00,837.00,800.00,706.00,595.00,809.00
4,2,Afghanistan,2905,Cereals - Excluding Beer,645,Food supply quantity (kg/capita/yr),kg,309.04,300.2,269.18,...,193.16,188.94,181.12,183.98,181.97,182.03,181.61,185.87,181.87,181.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58363,181,Zimbabwe,2769,"Aquatic Animals, Others",664,Food supply (kcal/capita/day),kcal/capita/day,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
58364,181,Zimbabwe,2928,Miscellaneous,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58365,181,Zimbabwe,2680,Infant food,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58366,181,Zimbabwe,2899,Miscellaneous,5123,Losses,1000 tonnes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Aggregating Food Suppy Quantity and Losses data and trimming Foody Supply data
food_balance_losses = filtered_food_balance_data[filtered_food_balance_data['Element Code'] == 5123].drop(['Item Code', 'Item'], axis=1)
food_balance_losses = food_balance_losses.groupby(by=['Area Code', 'Area', 'Element Code', 'Element', 'Unit']).sum().reset_index()
food_balance_losses.replace(0.0, np.nan, inplace=True)

food_balance_supply = filtered_food_balance_data[filtered_food_balance_data['Item Code'] == 2901]

food_balance_supply_qty = filtered_food_balance_data[filtered_food_balance_data['Element Code'] == 645].drop(['Item Code', 'Item'], axis=1)
food_balance_supply_qty = food_balance_supply_qty.groupby(by=['Area Code', 'Area', 'Element Code', 'Element', 'Unit']).sum().reset_index()
food_balance_supply_qty.replace(0.0, np.nan, inplace=True)

del food_balance_data_old
del food_balance_data_new




In [27]:
# Add the above aggregations to the mega dataset

mega_dataset = mega_dataset.append(food_balance_losses)
mega_dataset = mega_dataset.append(food_balance_supply)
mega_dataset = mega_dataset.append(food_balance_supply_qty).reset_index(drop=True)

In [28]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Item Code,Item
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.4,0.4,NaN,NaN,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN,NaN,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN,NaN,NaN
3,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.1,14.8,12.7,NaN,NaN,NaN
4,1,Armenia,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.2,1.1,1.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,255,Belgium,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1542.01,1781.34,1667.62,1720.14,1651.36,1902.92,1895.15,NaN,NaN,NaN
2150,256,Luxembourg,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1941.92,1857.42,1848.46,1896.22,1912.25,1906.44,1930.02,NaN,NaN,NaN
2151,272,Serbia,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,1459.05,1436.17,1469.79,1476.74,1454.39,1443.3,1462.18,NaN,NaN,NaN
2152,273,Montenegro,645,Food supply quantity (kg/capita/yr),kg,NaN,NaN,NaN,NaN,NaN,...,2085.65,2077.26,2169.48,2232.55,2246.37,2238.76,2250.12,NaN,NaN,NaN


In [29]:
# TODO: Add government expenditure data
expenditure_data = pd.read_csv('../data/Investment_GovernmentExpenditure_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")


In [30]:
filtered_expenditure_data = expenditure_data[
    (expenditure_data['Area Code'].isin(countries['Area Code']))
    & (expenditure_data['Element Code'] == 6111)
    & (expenditure_data['Item Code'] == 23161)
]
del expenditure_data

In [31]:
filtered_expenditure_data = filtered_expenditure_data.drop(labels=['Item Code', 'Item'], axis=1)


In [32]:
mega_dataset = mega_dataset.append(filtered_expenditure_data)

In [33]:
mega_dataset

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Item Code,Item
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.4,0.4,NaN,NaN,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN,NaN,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN,NaN,NaN
3,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.1,14.8,12.7,NaN,NaN,NaN
4,1,Armenia,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.2,1.1,1.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6525,236,Venezuela (Bolivarian Republic of),6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,1.16,4.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6577,237,Viet Nam,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,3.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6665,249,Yemen,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,0.79,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6732,251,Zambia,6111,Share of Total Expenditure,%,NaN,NaN,NaN,NaN,NaN,...,6.43,8.45,10.25,6.78,7.93,5.29,5.38,3.35,NaN,NaN


In [34]:
consumer_price_index_data = pd.read_csv('../data/ConsumerPriceIndices_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1")

In [35]:
filtered_consumer_price_index_data = consumer_price_index_data[
    (consumer_price_index_data['Item Code'] == 23014)
    & (consumer_price_index_data['Area Code'].isin(countries['Area Code']))
    & (consumer_price_index_data['Months'] == 'January')
]


In [36]:
filtered_consumer_price_index_data.sort_values('Area Code', inplace=True)
filtered_consumer_price_index_data.loc[:,'Element'] = filtered_consumer_price_index_data.loc[:,'Item']
filtered_consumer_price_index_data.loc[:,'Element Code'] = filtered_consumer_price_index_data.loc[:,'Item Code']
filtered_consumer_price_index_data = filtered_consumer_price_index_data.sort_values('Area Code')
filtered_consumer_price_index_data.drop(['Months', 'Months Code', 'Item', 'Item Code'], axis=1, inplace=True)

/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [37]:
filtered_consumer_price_index_data

,Area Code,Area,Unit,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Element,Element Code
348,1,Armenia,%,NaN,-5.093245,4.499744,2.559075,7.169667,9.205826,1.068394,...,3.941085,5.686249,-3.938860,1.803501,4.102667,0.487004,-1.743922,6.371823,Food price inflation,23014
24,2,Afghanistan,%,NaN,22.943765,11.612646,19.585062,8.592641,12.390806,10.635895,...,9.092442,2.957042,1.117175,4.698328,5.568586,-0.413473,5.985349,6.097938,Food price inflation,23014
96,3,Albania,%,NaN,7.093145,6.584784,5.926985,3.757915,-0.150075,-0.350701,...,2.650161,36.559391,-23.844752,8.480869,2.168390,4.346391,2.389434,1.178424,Food price inflation,23014
132,4,Algeria,%,NaN,-1.593977,4.138494,2.000131,4.502542,1.971861,-1.405532,...,1.397886,5.637749,1.989390,6.855657,1.815704,1.348250,-1.184074,3.948059,Food price inflation,23014
168,6,Andorra,%,NaN,2.899967,2.231815,4.690659,2.703747,1.833396,2.132767,...,1.590489,1.905590,1.210428,2.207912,2.520252,1.053556,1.998262,1.618399,Food price inflation,23014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,272,Serbia,%,NaN,86.345219,79.604262,8.595642,1.672241,11.918860,18.585285,...,-1.638478,0.483611,1.657754,1.209890,2.079002,2.953157,2.274975,0.435203,Food price inflation,23014
4368,273,Montenegro,%,NaN,2.217053,2.168966,2.122920,2.078789,2.036456,1.995812,...,-0.875843,1.786125,1.366027,1.226870,-0.517514,2.086028,2.653299,1.084081,Food price inflation,23014
6300,276,Sudan,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.774762,24.264132,9.913405,27.623577,56.211377,61.360641,67.735805,248.264690,Food price inflation,23014
6192,277,South Sudan,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.731684,-6.758713,198.672353,369.533890,112.550566,67.133998,47.882087,41.953464,Food price inflation,23014


In [38]:
mega_dataset.append(filtered_consumer_price_index_data)

,Area Code,Area,Element Code,Element,Unit,Y1950,Y1951,Y1952,Y1953,Y1954,...,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Item Code,Item,Y2021
0,1,Armenia,210081,Number of moderately or severely food insecure...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5,0.4,0.4,NaN,NaN,NaN,NaN
1,1,Armenia,210011,Number of people undernourished (million) (3-y...,millions,NaN,NaN,NaN,NaN,NaN,...,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,NaN,NaN,NaN,NaN
2,1,Armenia,210071,Number of severely food insecure people (milli...,millions,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<0.1,<0.1,<0.1,NaN,NaN,NaN,NaN
3,1,Armenia,210091,Prevalence of moderate or severe food insecuri...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,17.1,14.8,12.7,NaN,NaN,NaN,NaN
4,1,Armenia,210401,Prevalence of severe food insecurity in the to...,%,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.2,1.1,1.1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,272,Serbia,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,-1.638478,0.483611,1.657754,1.20989,2.079002,2.953157,2.274975,NaN,NaN,0.435203
4368,273,Montenegro,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,-0.875843,1.786125,1.366027,1.22687,-0.517514,2.086028,2.653299,NaN,NaN,1.084081
6300,276,Sudan,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,25.774762,24.264132,9.913405,27.623577,56.211377,61.360641,67.735805,NaN,NaN,248.264690
6192,277,South Sudan,23014,Food price inflation,%,NaN,NaN,NaN,NaN,NaN,...,-6.731684,-6.758713,198.672353,369.53389,112.550566,67.133998,47.882087,NaN,NaN,41.953464
